In [4]:
import pandas as pd
import numpy as np
import time
from datetime import datetime

--- NumPy версія ---

In [5]:
print("=== NumPy версія ===")

# Типи даних для кожної колонки
types = [
    ("Date", "U10"),
    ("Time", "U8"),
    ("Global_active_power", "f8"),
    ("Global_reactive_power", "f8"),
    ("Voltage", "f8"),
    ("Global_intensity", "f8"),
    ("Sub_metering_1", "f8"),
    ("Sub_metering_2", "f8"),
    ("Sub_metering_3", "f8")
]

# Завантаження даних
start = time.time()
df_np = np.genfromtxt("household_power_consumption.txt",
                      missing_values=["?", ""],
                      delimiter=';',
                      dtype=types,
                      encoding="UTF-8",
                      names=True)
df_np = df_np[~np.isnan(df_np['Global_active_power'])]
end = time.time()
print("Час завантаження та очищення NumPy:", round(end - start, 4), "сек")

# 1. Sub_metering_2 > Sub_metering_3
start = time.time()
selected_np = df_np[df_np['Sub_metering_2'] > df_np['Sub_metering_3']]
end = time.time()
print(" NumPy: Sub_metering_2 > Sub_metering_3:", round(end - start, 4), "сек")

# 2. Випадкові 5000 записів
start = time.time()
random_np = df_np[np.random.choice(len(df_np), size=5000, replace=False)]
end = time.time()
print("NumPy: 5000 випадкових записів:", round(end - start, 4), "сек")

# 3. Складна умова
start = time.time()
complex_np = df_np[(df_np['Sub_metering_1'] > 1.0) & (df_np['Voltage'] > 240)]
end = time.time()
print("NumPy: Складна умова:", round(end - start, 4), "сек")

# 4. Вибір колонок
start = time.time()
columns_np = df_np[['Voltage', 'Global_intensity']]
end = time.time()
print("NumPy: Вибір колонок:", round(end - start, 4), "сек")

# 5. Потужність > 5 кВт
start = time.time()
power_np = df_np[df_np['Global_active_power'] > 5]
end = time.time()
print("NumPy: Global_active_power > 5 кВт:", round(end - start, 4), "сек")

# 6. Напруга > 235 В
start = time.time()
volt_np = df_np[df_np['Voltage'] > 235]
end = time.time()
print("NumPy: Voltage > 235 В:", round(end - start, 4), "сек")

# 7. Потужність 19-20А, група 2 найбільша
start = time.time()
current_np = df_np[(df_np['Global_intensity'] >= 19) & (df_np['Global_intensity'] <= 20)]
cond_np = current_np[(current_np['Sub_metering_2'] > current_np['Sub_metering_1']) & 
                     (current_np['Sub_metering_2'] > current_np['Sub_metering_3'])]
end = time.time()
print("NumPy: Потужність 19-20А, група 2 найбільша:", round(end - start, 4), "сек")

# 8. Вибірка 500000 записів, середнє
start = time.time()
sample_np = df_np[np.random.choice(len(df_np), size=500000, replace=False)]
means_np = np.array([
    np.mean(sample_np['Sub_metering_1']),
    np.mean(sample_np['Sub_metering_2']),
    np.mean(sample_np['Sub_metering_3'])
])
end = time.time()
print(" NumPy: 500000 записів, середнє:", round(end - start, 4), "сек")
print(" Середні значення NumPy:", means_np)

# 9. Після 18:00, >6 кВт, група 2 найбільша, кожен 3-й/4-й
start = time.time()
times_np = np.array([datetime.strptime(t, "%H:%M:%S").time() for t in df_np['Time']])
after_18_mask = np.array([t >= datetime.strptime("18:00:00", "%H:%M:%S").time() for t in times_np])
filtered_np = df_np[(df_np['Global_active_power'] > 6) & after_18_mask]
group2_mask = (filtered_np['Sub_metering_2'] > filtered_np['Sub_metering_1']) & \
              (filtered_np['Sub_metering_2'] > filtered_np['Sub_metering_3'])
filtered_group2_np = filtered_np[group2_mask]
half_np = len(filtered_group2_np) // 2
first_half_np = filtered_group2_np[:half_np]
second_half_np = filtered_group2_np[half_np:]
final_np_selection = np.concatenate([first_half_np[::3], second_half_np[::4]])
end = time.time()
print("NumPy: після 18:00, >6кВт, група 2 max, кожен 3-й/4-й:", round(end - start, 4), "сек")


=== NumPy версія ===
Час завантаження та очищення NumPy: 18.7428 сек
 NumPy: Sub_metering_2 > Sub_metering_3: 0.0485 сек
NumPy: 5000 випадкових записів: 0.0853 сек
NumPy: Складна умова: 0.0294 сек
NumPy: Вибір колонок: 0.01 сек
NumPy: Global_active_power > 5 кВт: 0.0235 сек
NumPy: Voltage > 235 В: 0.1981 сек
NumPy: Потужність 19-20А, група 2 найбільша: 0.031 сек
 NumPy: 500000 записів, середнє: 0.2481 сек
 Середні значення NumPy: [1.116178 1.292822 6.44049 ]
NumPy: після 18:00, >6кВт, група 2 max, кожен 3-й/4-й: 49.7484 сек


 --- Pandas версія ---

In [6]:
print("\n=== Pandas версія ===")

# Завантаження
start = time.time()
pandas_df = pd.read_csv("household_power_consumption.txt", sep=';', na_values=['?'])
pandas_df.dropna(inplace=True)
end = time.time()
print("Час завантаження та очищення Pandas:", round(end - start, 4), "сек")

# 1
start = time.time()
selected = pandas_df[pandas_df['Sub_metering_2'] > pandas_df['Sub_metering_3']]
end = time.time()
print("Pandas: Sub_metering_2 > Sub_metering_3:", round(end - start, 4), "сек")

# 2
start = time.time()
random_sample = pandas_df.sample(n=5000, replace=False)
end = time.time()
print("Pandas: 5000 випадкових записів:", round(end - start, 4), "сек")

# 3
start = time.time()
complex_selection = pandas_df[(pandas_df['Sub_metering_1'] > 1.0) & (pandas_df['Voltage'] > 240)]
end = time.time()
print("Pandas: Складна умова:", round(end - start, 4), "сек")

# 4
start = time.time()
selected_columns = pandas_df[['Voltage', 'Global_intensity']]
end = time.time()
print("Pandas: Вибір колонок:", round(end - start, 4), "сек")

# 5
start = time.time()
power_gt_5 = pandas_df[pandas_df['Global_active_power'] > 5]
end = time.time()
print("Pandas: Global_active_power > 5 кВт:", round(end - start, 4), "сек")

# 6
start = time.time()
voltage_gt_235 = pandas_df[pandas_df['Voltage'] > 235]
end = time.time()
print("Pandas: Voltage > 235 В:", round(end - start, 4), "сек")

# 7
start = time.time()
current_range = pandas_df[(pandas_df['Global_intensity'] >= 19) & (pandas_df['Global_intensity'] <= 20)]
condition_groups = current_range[(current_range['Sub_metering_2'] > current_range['Sub_metering_1']) &
                                 (current_range['Sub_metering_2'] > current_range['Sub_metering_3'])]
end = time.time()
print("Pandas: Потужність 19-20А, група 2 найбільша:", round(end - start, 4), "сек")

# 8
start = time.time()
sample_500k = pandas_df.sample(n=500000, replace=False)
means = sample_500k[['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']].mean()
end = time.time()
print("Pandas: 500000 записів, середнє:", round(end - start, 4), "сек")
print("Середні значення Pandas:\n", means)

# 9
start = time.time()
pandas_df['Time'] = pd.to_datetime(pandas_df['Time'], format='%H:%M:%S', errors='coerce')
filtered_time = pandas_df[(pandas_df['Time'].dt.hour >= 18) & (pandas_df['Global_active_power'] > 6)]
filtered_group2 = filtered_time[(filtered_time['Sub_metering_2'] > filtered_time['Sub_metering_1']) &
                                (filtered_time['Sub_metering_2'] > filtered_time['Sub_metering_3'])]
half = len(filtered_group2) // 2
first_half = filtered_group2.iloc[:half]
second_half = filtered_group2.iloc[half:]
final_selection = pd.concat([first_half.iloc[::3], second_half.iloc[::4]])
end = time.time()
print("Pandas: після 18:00, >6кВт, група 2 max, кожен 3-й/4-й:", round(end - start, 4), "сек")


=== Pandas версія ===
Час завантаження та очищення Pandas: 2.923 сек
Pandas: Sub_metering_2 > Sub_metering_3: 0.0434 сек
Pandas: 5000 випадкових записів: 0.0899 сек
Pandas: Складна умова: 0.0125 сек
Pandas: Вибір колонок: 0.0123 сек
Pandas: Global_active_power > 5 кВт: 0.0069 сек
Pandas: Voltage > 235 В: 0.1487 сек
Pandas: Потужність 19-20А, група 2 найбільша: 0.0099 сек
Pandas: 500000 записів, середнє: 0.2194 сек
Середні значення Pandas:
 Sub_metering_1    1.118262
Sub_metering_2    1.301106
Sub_metering_3    6.462118
dtype: float64
Pandas: після 18:00, >6кВт, група 2 max, кожен 3-й/4-й: 7.5154 сек
